# Deploy VARCO LLM BASE 2.0 Algorithm from AWS Marketplace 




VARCO LLM 2.0 is NCSOFT's large language model that can be applied to the development of various natural language processing-based AI services 
such as text generation, question answering, chatbots, summarization, and information extraction.
NCSOFT's VARCO LLM 2.0 was developed with our own technology, including data construction, pre-training, instruction tuning and alignment tuning.
We evaluated VARCO LLM 2.0 on various NLP tasks and its performance has significantly improved compared to VARCO LLM 1.0, and it boasts the highest performance among other Korean LLMs of similar sizes. 
In particular, it has been trained to be used in high-level natural language processing applications such as creative writing, summarization, question and answering, chatbots and translation, and shows high performance in related quantitative indicators.
For inquiries regarding further performance improvement or collaboration for service applications, please contact us by email (varco_llm@ncsoft.com).

This sample notebook shows you how to deploy [varco llm](https://aws.amazon.com/marketplace/ai/procurement?productId=e2a2cbf9-1862-42bc-946c-5437ba511026)  using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  

## Contents:
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
2. [Set up environment](#2.-Set-up-environment)
3. [Train a model](#3.-Train-a-model)
4. [Create an endpoint and perform real-time inference](#4.-Create-an-endpoint-and-perform-real-time-inference)
4. [Clean-up](#5.-Clean-up)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm [listing page](https://aws.amazon.com/marketplace/ai/procurement?productId=e2a2cbf9-1862-42bc-946c-5437ba511026)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## 2. Set up environment

In [ ]:
algo_arn = "arn:aws:sagemaker:us-west-2:594846645681:algorithm/varco-llm-base-ist-948f1c4489f7326a8bc0fcb6801cffcd"

In [ ]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import numpy as np
import io
from sagemaker.algorithm import AlgorithmEstimator

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")

my_instance_type = "ml.g5.12xlarge"

content_type = "application/json"

my_seed = 42

## 3. Train a model

In [ ]:
training_job_name = "varcobaseTJ"
algo = AlgorithmEstimator(
    algorithm_arn=algo_arn,
    role=role,
    instance_count=1,
    instance_type=my_instance_type,
    sagemaker_session=sagemaker_session,
    base_job_name=training_job_name,
    train_volume_size=300,
)

In [ ]:
inputs = {"train": "s3://sagemaker-us-west-2-973735099617/finetune/training_data/"}

algo.fit(inputs)

## 4. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A.Create an endpoint

Once endpoint has been created, you would be able to perform real-time inference.

In [ ]:
my_endpoint_name = "varcobaseEP"
predictor = algo.deploy(
    initial_instance_count=1,
    instance_type=my_instance_type,
    endpoint_name=my_endpoint_name
)

### B.Create input payload

In [ ]:
input = {
    "repetition_penalty": 1.05,
    "temperature": 0.3,
    "top_k": 50,
    "top_p": 1,
    "text": "안녕"
}

### C. Perform real-time inference

#### C-1. Streaming

In [ ]:
class VarcoInferenceStream():
    def __init__(self, sagemaker_runtime, endpoint_name):
        self.sagemaker_runtime = sagemaker_runtime
        self.endpoint_name = endpoint_name

    def stream_inference(self, request_body):
        # Gets a streaming inference response
        # from the specified model endpoint:
        response = self.sagemaker_runtime\
            .invoke_endpoint_with_response_stream(
                EndpointName=self.endpoint_name,
                Body=json.dumps(request_body),
                ContentType="application/json"
        )
        # Gets the EventStream object returned by the SDK:
        for body in response["Body"]:
            raw = body['PayloadPart']['Bytes']
            yield raw.decode()


sm_runtime = boto3.client("sagemaker-runtime")
varco_inference_stream = VarcoInferenceStream(sm_runtime, my_endpoint_name)
stream = varco_inference_stream.stream_inference(input)
for part in stream:
    print(part, end='')

## 5. Clean-up

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()